# How to determine which parcellation is best?

The goal of this notebook will be relatively straightforward, essentially we want to ask the question, how does different choice of parcellation influence performance? This example notebook assumes that we have already run the set of pre-processing steps avaliable in prepare_frmi_data.ipynb.

- How do you know which steps should be divided into front-end pre-processing and which steps should be nested in cross-validation?

As a rule of thumb, you can distinguish between steps that operate on a single individuals data and those that require information across multiple subjects. Essentially, any step or transformation that can be computed and applied on the level of the individual, it is fine to do these steps ahead of time. On the otherhand, any steps, like column-wise standardization / z-score, that involve information as computed across a group of subjects, should be down in a properly nested manner / as nested within a machine learning pipeline.





Broader topic is: How to structure and answer a question of interest within a predictive framework?

This can be the general structure of the chapter, weaving in this specific example problem.
In terms of specifics, let's perform a global train-test split, then on the training set investigate different choices of ML model / connectivity measure, then use the Test split to answer our main topic of interest, the different parcellations.


- Broadly introduce what a predictive framework is

- Examples on how to frame questions in terms of prediction, e.g., basic, to more clever setups, so that you can answer your question based on the difference in performance from different setups.

- Introduce the actual example we will use here + any other relevant background. Mntion the extra-preprocessing / preperation / link the notebook.

- The importance of generalizability, w/ example of pitfall, i.e., training accuracy

- Note the relativity of  estimating generalizability, or of the generalizability one might care about, as related to common neuro-imaging datasets, e.g., maybe you care about generalizing to new unseen sites or people from a different country, or maybe you don't, and how you can build that into 

- Techniques to estimate generalizability - Train / Test, K-Fold CV

- General structure of an ML predictive pipeline, essentially designed to be used in context where estimating generalizability is desired

- Then more specific structure / choices of an ML predictive pipeline for this question / task based connectivity, at the start of this section, 

- How do you know which steps should be divided into front-end pre-processing and which steps should be nested in cross-validation?

- Choice of cross-validation technique as related to main question of interest / pitfalls of CV

- Neuroimaging specific things, like confounds, and *briefly* introduce some different ways of 1. Detecting the influence of potential confounds, 2. Trying to correct for their influence. 

- Feature importances / the difficulty in making sense of feature importances when the underlying data is complex like with task connectivity. Make a just terrible overwhelming visualization of 10k connections on a brain.

- Limitations of a predictive framework (correlation not causation, just because one method is not predictive doesn't mean a different method isn't)

TODO: Put this piece later on, mixed in which the real examples~

## Important conceptual note on the choice of cross-validation technique as related to main question of interest

In this notebook our main question of interest is the comparison of performance as broken down by parcellation. This is important because it directly influences our choices in how we setup our cross-validation. Specifically, we will be performing 5-fold cross validation (CV) in this notebook across the entire set of avaliable subjects. This is an okay choice because ultimately every single set of results from every 5-fold CV we run are a part of our main results of interest. This is an important point! 

It is likely easier to think about this concept in terms of an example on what you are not supposed to do. Let's say instead our question of interest wasn't related to the parcellation, but instead on how well we could predict a certain phenotype, let's say age, from our task connectivity. Then, as a sub question maybe we have the same setup as we do here, 6 different parcellations from a mix of volumetric and surface based avaliable, so we proceed in exactly the same way and try all 6 different parcellations with 5-fold cross validation on the full set of avaliable subjects. Then, we take the one that did best and then move on from there just using that parcellation, and we dig into those results, examine the feature importances, and report just those results, for that parcellation. So why is that wrong? Essentially, because cross-validation isn't fullproof. Consider another example, where we do the same thing, but this time we have 100,000 randomly generated parcellations. We go through and test every single one, and 6 months later when everything is finnally done running, we find one of those parcellations has done amazing! So we say great, and just report those results, even though now its becoming more and more clear that this result was almost certainly a result of dumb luck.

The core issue in both cases is that these results would be presented with no gaurentee of generalizability, granted to different degrees. I think for some reason especially when youn are first getting started with machine learning, it is too easy to think of cross-validation techniques, like K-fold, as a silver bullet. When really, these estimates you get out from them are potentially noisy, especially at smaller sample sizes! What this comes down to is keeping generalizability in mind.

That is to say, in practice, you must make sure that any "hyper-parameters" within the machine learning pipeline (in the example here, choice of parcellation is a hyper-parameter, but these broadly refer to ANY choice which is not directly a part your research question) is not informed by the set of data ultimately used to estimate generalizability (either a test set, or the validation sets in a K-Fold setup). Explicitly for each 'hyper-parameter' you have the following options:

- If this parameter is important to your research topic, both test and report the results by each possible value of interest that parameter might take.

- Otherwise, if not important / directly related, you have a few options:
    1. Fix the value ahead of time based on some aprioi knowledge or guess.
    2. Assign the value through nested cross-validation (e.g., train-val-test split or nested K-fold)

So for example, if my question of interest included: "what's better a Random Forest or a Ridge Regression?", it would be desirable to test both of these estimators directly, but if it wasn't, then we would want to either just fix the choice ahead of time. Let's say we know ridge regression has worked well in the past on simmilar problems, so we can just choose to use it. Or, we could take the tact of assigning it through nested cross validation. Now this second option has plently of different variations. For example, one way would be to split our original training dataset into a further training and validation dataset, then to test each model on the validation set and use that choice to inform our final choice, which we would ultimatly test on the test set. Another variation would be, within the training set, perform say 5-Fold cross validation, and whichever model gets the highest average score, select that one. 

This concept 

First, we load our prepared BPt dataset.

In [44]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import BPt as bp
import os
from os.path import dirname, abspath

In [45]:
# Useful directories
main_dr = dirname(abspath(os.getcwd()))
data_dr = os.path.join(main_dr, 'data')

# This is optional, but speeds up some
# operations, to ignore, set to None
cache_loc = os.path.join(data_dr, 'cache', 'fmri')

# Load in our pre-saved dataset
data = bp.read_pickle(os.path.join(data_dr, 'datasets', 'fmri.dataset'))
data

,basc_444,difumo_256,gordon,hcp_mmp,juelich,schaefer_400
participant_id,,,,,,
sub-0001,Loc(4901),Loc(3139),Loc(2264),Loc(522),Loc(4020),Loc(1393)
sub-0002,Loc(5143),Loc(3381),Loc(2504),Loc(762),Loc(4262),Loc(1633)
sub-0003,Loc(4640),Loc(2878),Loc(2005),Loc(263),Loc(3759),Loc(1134)
sub-0004,Loc(5155),Loc(3393),Loc(2516),Loc(774),Loc(4274),Loc(1645)
sub-0005,Loc(4668),Loc(2906),Loc(2033),Loc(291),Loc(3787),Loc(1162)
...,...,...,...,...,...,...
sub-0923,Loc(4721),Loc(2959),Loc(2086),Loc(344),Loc(3840),Loc(1215)
sub-0924,Loc(5150),Loc(3388),Loc(2511),Loc(769),Loc(4269),Loc(1640)
sub-0925,Loc(4794),Loc(3032),Loc(2158),Loc(416),Loc(3913),Loc(1287)


In [46]:
data = data.set_test_split(.2)
data

Performing test split on: 871 subjects.
random_state: None
Test split size: 0.2

Performed train/test split
Train size: 696
Test size:  175


,basc_444,difumo_256,gordon,hcp_mmp,juelich,schaefer_400
participant_id,,,,,,
sub-0001,Loc(4901),Loc(3139),Loc(2264),Loc(522),Loc(4020),Loc(1393)
sub-0002,Loc(5143),Loc(3381),Loc(2504),Loc(762),Loc(4262),Loc(1633)
sub-0003,Loc(4640),Loc(2878),Loc(2005),Loc(263),Loc(3759),Loc(1134)
sub-0004,Loc(5155),Loc(3393),Loc(2516),Loc(774),Loc(4274),Loc(1645)
sub-0005,Loc(4668),Loc(2906),Loc(2033),Loc(291),Loc(3787),Loc(1162)
...,...,...,...,...,...,...
sub-0923,Loc(4721),Loc(2959),Loc(2086),Loc(344),Loc(3840),Loc(1215)
sub-0924,Loc(5150),Loc(3388),Loc(2511),Loc(769),Loc(4269),Loc(1640)
sub-0925,Loc(4794),Loc(3032),Loc(2158),Loc(416),Loc(3913),Loc(1287)


In [142]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFpr, f_regression, f_classif
from nilearn.connectome import ConnectivityMeasure
import numpy as np
import BPt as bp

class CPMEdgeSum(BaseEstimator, TransformerMixin):
    
    def __init__(self, sum_type='all'):
        self.sum_type = sum_type
    
    def fit(self, X, y):
        pass
    
    def transform(self, X, y=None):
        
        def sum_x(x):
            return np.expand_dims(x.sum(axis=-1), axis=-1)
        
        if self.sum_type == 'all':
            return sum_x(X)
        
        elif self.sum_type == 'pos':
            X_temp = X.copy()
            X_temp[X<0] = 0
            return sum_x(X_temp)
        
        elif self.sum_type == 'neg':
            X_temp = X.copy()
            X_temp[X>0] = 0
            return sum_x(X_temp)
        
        elif self.sum_type == 'both':
            
            X_temp = X.copy()
            X_temp[X<0] = 0
            pos = sum_x(X_temp)
            
            X_temp = X.copy()
            X_temp[X>0] = 0
            neg = sum_x(X_temp)

            return np.hstack([pos, neg])

def get_cpm_trans(sum_type):
    return bp.Transformer(CPMEdgeSum(sum_type))


def get_cpm_pipe(alpha=0.05, sum_type='pos',
                 model_str='linear', is_binary=False):

    # Make a custom feature selector, p-val selection at alpha
    scoring = f_classif
    if is_binary:
        scoring = f_regression

    fs = bp.FeatSelector(SelectFpr(scoring, alpha=alpha))
    
    # Get CPM style sum
    cpm_trans = get_cpm_trans(sum_type)
    
    # Use base correlation connectivity
    con = ConnectivityMeasure(kind='correlation',
                              discard_diagonal=True,
                              vectorize=True)
    corr_loader = bp.Loader(con, behav='all', cache_loc=cache_loc)
    
    # Use a just linear model
    model = bp.Model(model_str)
    
    # Return as pipeline
    return bp.Pipeline([corr_loader, fs, cpm_trans, model])

In [143]:
pipe = get_cpm_pipe(alpha=bp.Compare([.05, .01]),
                    sum_type=bp.Compare(['pos', 'neg', 'all', 'both']),
                    model_str=bp.Compare(['linear', 'svm']),
                    is_binary=False)

results = bp.evaluate(pipe, data,
                      scope='basc_444', target='age',
                      subjects='train', cv=5, 
                      eval_verbose=0)

results.summary()

Compare:   0%|          | 0/16 [00:00<?, ?it/s]

Folds:   0%|          | 0/100 [00:00<?, ?it/s]

,,,mean_scores_explained_variance,mean_scores_neg_mean_squared_error,std_scores_explained_variance,std_scores_neg_mean_squared_error
steps__1__obj__alpha,steps__2__obj__sum_type,steps__3__obj,,,,
0.05,pos,linear,-0.004325,-2.899389,0.009576,0.981367
0.01,pos,linear,-0.004721,-2.900978,0.006987,0.982426
0.05,neg,linear,0.001763,-2.890226,0.045146,0.986089
0.01,neg,linear,0.002721,-2.888502,0.048697,0.986772
0.05,all,linear,-0.003993,-2.899643,0.014054,0.982777
0.01,all,linear,-0.003851,-2.899923,0.021985,0.984309
0.05,both,linear,-0.000962,-2.899290,0.051597,0.986009
0.01,both,linear,-0.000603,-2.899655,0.053081,0.987766
0.05,pos,svm,-0.049992,-2.997322,0.123344,1.050151


In [136]:
from BPt.default.pipelines import ridge_pipe

con = ConnectivityMeasure(kind='correlation',
                          discard_diagonal=True,
                          vectorize=True)
corr_loader = bp.Loader(con, behav='all', cache_loc=cache_loc)

pipe = bp.Pipeline([corr_loader, ridge_pipe])

results = bp.evaluate(pipe, data,
                      scope='basc_444', target='age',
                      subjects='train', cv=5, 
                      eval_verbose=0)
results

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

BPtEvaluator
------------
explained_variance: 0.0303 ± 0.0190
neg_mean_squared_error: -2.80 ± 0.2915

Saved Attributes: ['estimators', 'preds', 'timing', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Avaliable Methods: ['to_pickle', 'compare', 'get_X_transform_df', 'get_inverse_fis', 'get_preds_dfs', 'subset_by', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: age
problem_type: regression
scope: basc_444
subjects: train
random_state: 1


In [137]:
con = ConnectivityMeasure(kind='tangent',
                          discard_diagonal=True,
                          vectorize=True)
loader = bp.Loader(con, behav='all', cache_loc=cache_loc)

pipe = bp.Pipeline([loader, ridge_pipe])

results = bp.evaluate(pipe, data,
                      scope='basc_444', target='age',
                      subjects='train', cv=5, 
                      eval_verbose=0)
results

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

BPtEvaluator
------------
explained_variance: 0.0759 ± 0.0105
neg_mean_squared_error: -2.67 ± 0.3123

Saved Attributes: ['estimators', 'preds', 'timing', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Avaliable Methods: ['to_pickle', 'compare', 'get_X_transform_df', 'get_inverse_fis', 'get_preds_dfs', 'subset_by', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: age
problem_type: regression
scope: basc_444
subjects: train
random_state: 1


In [ ]:
from nilearn.connectome import ConnectivityMeasure

compare_target = bp.Compare(data.get_cols('target'))
compare_scope = bp.Compare(['difumo_256', 'gordon'])



ps = bp.ProblemSpec(target=compare_target, 
                    scope=compare_scope,
                    n_jobs=6,
                    random_state=1
                   )

# This is the base object responsible for generating the correlation matrix
con = ConnectivityMeasure(kind='tangent',
                          discard_diagonal=True,
                          vectorize=True)

# We just need to wrap it in a BPt Loader object
loader = bp.Loader(con, behav='all', cache_loc=cache_loc)

# The base model we will try is ridge regression
# with a random search for hyper-parameters over different params
ridge_rs = bp.Model('ridge', params=1,
                    param_search=bp.ParamSearch(n_iter=30))
elastic_rs = bp.Model('elastic', params=1,
                      param_search=bp.ParamSearch(n_iter=60))
svm_rs = bp.Model('svm', params=1,
                  param_search=bp.ParamSearch(n_iter=60))

# Wrap in compare
compare_model = bp.Compare([bp.Option(ridge_rs, 'ridge')
                            bp.Option(elastic_rs, 'elastic'),
                            bp.Option(svm_rs, 'svm')])

# Put the steps together in a pipeline
pipe = bp.Pipeline([loader, compare_model], cache_loc=cache_loc)

# Evaluate
results = bp.evaluate(pipe, data, ps, mute_warnings=True)

results.summary()

To begin, we will fix the target variable of interest as BMI, and evaluate using just the gordon parcellation. These parameters we can specify in the parameter holding ProblemSpec class.

In [ ]:
ps = bp.ProblemSpec(target='BMI', 
                    scope='gordon', # This defines the subset of columns to use
                    n_jobs=6,
                    random_state=1
                   )

Notably, our raw data is still in the form of time-series which by default are not in a format that most ML models can accept. The first step of our pipeline will therefore be to transform these ROI timeseries in some way. The first method we will try is the most basic / common in which we generate a correlation matrix between each ROI's timeseries.

In [ ]:
from nilearn.connectome import ConnectivityMeasure

# This is the base object responsible for generating the correlation matrix
con = ConnectivityMeasure(kind='correlation',
                          discard_diagonal=True,
                          vectorize=True)

# We just need to wrap it in a BPt Loader object
loader = bp.Loader(con, behav='all', cache_loc=None)

# The base model we will try is ridge regression
# with a random search for hyper-parameters over different params
ridge_rs = bp.Model('ridge', params=1,
                    param_search=bp.ParamSearch(n_iter=10))

# Put the steps together in a pipeline
pipe = bp.Pipeline([loader, ridge_rs], cache_loc=cache_loc)

# Evaluate
results = bp.evaluate(pipe, data, ps, mute_warnings=True)
results

The next variation we will try is employing tangent connectivity instead of the base correlations.

In [ ]:
# Repeat the same steps as before, but this time using kind='tangent'
con = ConnectivityMeasure(kind='tangent',
                          discard_diagonal=True,
                          vectorize=True)
loader = bp.Loader(con, behav='all', cache_loc=cache_loc)

# Replace model in pipeline
pipe = bp.Pipeline([loader, ridge_rs], cache_loc=cache_loc)

# Evaluate
results = bp.evaluate(pipe, data, ps, mute_warnings=True)
results

Not bad, just by changing the "kind" of connectivity, we make a big jump in performance.

What if we want to try a different parcellation? Well all we have to do is change the scope.

In [ ]:
# These are our options
parcs = list(data['data'])

parcs

In [ ]:
# Change the scope to be a compare object
ps.scope = bp.Compare(parcs)

# Now re-run evaluate, but this time there is an extra compare
# loop which will run the same 5-fold evaluation, but for all
# parcellations seperately.
results = bp.evaluate(pipe, data, ps, mute_warnings=True)
results.summary()